# 🧑‍🏫 Clean Architecture en Python — Modèle **3 couches**

**Couches :** `Presentation` → `Domain` ← `Data`

Ce notebook illustre une architecture simple en 3 couches :

- **Presentation** : API/CLI/UI (ne contient pas de logique métier)
- **Domain** : règles métier + use cases (indépendant des frameworks et de la BDD)
- **Data** : accès aux ressources et aux données (implémentations concrètes des repositories)

👉 Objectifs :
1. Comprendre le rôle de chaque couche
2. Implémenter un petit cas d'usage (récupération d'une liste de produits)
3. Tester les use cases **sans** dépendre des données
4. Montrer qu'on peut **échanger** la Data layer sans toucher au Domain


## 0) Code "anti-pattern" (à ne **pas** reproduire)

On mélange tout : logique métier, accès aux données, et I/O.

In [1]:
# ⚠️ Exemple volontairement mauvais : tout en un seul endroit
all_products = [
    {"id": 1, "name": "Keyboard", "price": 29.9},
    {"id": 2, "name": "Mouse", "price": 19.9},
    {"id": 3, "name": "Monitor", "price": 149.0},
]

def list_products():
    print("[DB] Connexion à la base...")  # mélange I/O + accès “données”
    products = list(all_products)        # accès direct aux “données” globales

    print(f"[IO] Renvoi de {len(products)} produit(s)...")  # I/O dans la même fonction
    return products

# Appel “présentation” (ici, on imprime directement) dans le même module
result = list_products()
print(result)

[DB] Connexion à la base...
[IO] Renvoi de 3 produit(s)...
[{'id': 1, 'name': 'Keyboard', 'price': 29.9}, {'id': 2, 'name': 'Mouse', 'price': 19.9}, {'id': 3, 'name': 'Monitor', 'price': 149.0}]


## 1) Structure cible (3 couches)

```
project/
 ├─ di.py                     # Container Injection de dépendances : fournit le repo (InMemory, DB...)
 ├─ main.py                   # Point d’entrée de l’application (import app et lance Uvicorn/FastAPI)
 ├─ presentation/             # Couche Presentation : interface avec le monde extérieur (API, CLI, UI...)
 │   └─ api.py                # Définit l'objet FastAPI et les routes → appelle les Use Cases en injectant le repo
 ├─ domain/                   # Couche Domain : cœur métier, indépendant des frameworks
 │   ├─ entities.py           # Définition des entités métier (ex: Product)
 │   ├─ use_cases.py          # Cas d’usage (application logic) → manipule les entités via les interfaces
 │   └─ interfaces.py         # Interfaces (protocols) : définit les contrats que doit respecter la couche Data
 └─ data/                     # Couche Data : implémentations concrètes des interfaces définis dans Domain
     ├─ in_memory_repo.py     # Repo “faux” en mémoire (utile pour les tests/démo)
     └─ db_repo.py            # Repo “réel” connecté à une base de données (SQLite, Postgres, etc.)
```

**Règle d'or des dépendances (pointent vers le *Domain*) :**
- ✅ `Presentation → Domain`
- ✅ `Data → Domain`
- ❌ **Aucune dépendance du Domain vers Presentation/Data**
- ❌ **Aucune dépendance directe Presentation ↔ Data** (elles communiquent via le Domain)

Schéma :

![Clean Architecture](img/clean_architecture.png)

La **Presentation** dépend du **Domain** (appel des use cases). Le **Domain** ne connaît que des **interfaces**. La **Data** implémente ces interfaces.


## 2) Domain Layer — Entités, Ports, Use Cases

Le domaine ne doit dépendre ni de FastAPI, ni de SQLAlchemy, ni d'aucun framework.

In [2]:
# domain/entities.py
from dataclasses import dataclass

@dataclass()
class Product:
    id: int
    name: str
    price: float

In [3]:
# domain/interfaces.py — contrats que le Data layer doit respecter
from typing import List
from abc import ABC, abstractmethod

class ProductRepository(ABC):
    @abstractmethod
    def get_all_products(self) -> List[Product]:
        """Retourne tous les produits disponibles"""
        pass

In [4]:
# domain/use_cases.py
from typing import List, Dict

class GetProductsUseCase:
    """Use case pour récupérer la liste des produits."""
    def __init__(self, repo: ProductRepository):
        self.repo = repo

    def execute(self) -> List[Dict]:
        return [
            {"id": p.id, "name": p.name, "price": p.price}
            for p in self.repo.get_all_products()
        ]

## 3) Data Layer — Implémentations concrètes des interfaces/contrats

On fournit **une** implémentation en mémoire (simple et testable).

In [5]:
# data/in_memory_repo.py
from typing import List

class InMemoryProductRepository(ProductRepository):
    """Implémentation en mémoire (exemple statique) du ProductRepository."""
    
    def get_all_products(self) -> List[Product]:
        return [
            Product(1, "Keyboard", 29.9),
            Product(2, "Mouse", 19.9),
            Product(3, "Monitor", 149.0),
        ]

Une implémentation d'une réelle BDD ou d'un service est envisageable, même exigée dans un contexte réel.

In [6]:
# data/db_repo.py
from typing import List
from time import sleep

class DbProductRepository(ProductRepository):
    """Simule un repository qui irait chercher dans une base de données."""

    def get_all_products(self) -> List[Product]:
        print("[DB] Connexion à la base...")
        sleep(3)  # petite latence pour simuler l'accès DB
        print("[DB] SELECT * FROM products;")

        # On retourne des données comme si elles venaient d'une vraie table
        return [
            Product(1, "Keyboard", 29.9),
            Product(2, "Mouse", 19.9),
            Product(3, "Monitor", 149.0),
        ]

## 4) Presentation Layer — Exposer les use cases

In [10]:
# presentation/main.py — démo ultra simple (produits)
from pprint import pprint

repository = InMemoryProductRepository()
#repository = DbProductRepository()
get_products_use_case = GetProductsUseCase(repo=repository)

print("Produits disponibles :")
pprint(get_products_use_case.execute())

Produits disponibles :
[{'id': 1, 'name': 'Keyboard', 'price': 29.9},
 {'id': 2, 'name': 'Mouse', 'price': 19.9},
 {'id': 3, 'name': 'Monitor', 'price': 149.0}]


## 5) Unit Tests — Le Domain est testable **sans** Data réelle

In [8]:
# tests/test_use_cases.py — version simplifiée "produits"

# Fake repo minimal conforme à l'interface ProductRepository (get_all_products uniquement)
class FakeProductRepo(ProductRepository):

    def get_all_products(self):
        # Renvoie des objets ayant id/name/price (comme le port le demande)
        return [
            Product(id=1, name="Keyboard", price=29.9),
            Product(id=2, name="Mouse",    price=19.9),
            Product(id=3, name="Monitor",  price=149.0)
        ]

# Use case à tester
def test_get_products_returns_all():
    repo = FakeProductRepo()
    uc = GetProductsUseCase(repo)

    products = uc.execute()  # doit renvoyer une liste de dicts sérialisables
    assert isinstance(products, list)
    assert len(products) == 3

    # Vérifie le mapping / champs
    p1 = products[0]
    assert set(p1.keys()) == {"id", "name", "price"}
    assert p1["id"] == 1
    assert p1["name"] == "Keyboard"
    assert isinstance(p1["price"], (int, float))

print("✅ Tests produits OK")

✅ Tests produits OK


## 6) Checklist
- [ ] La couche Domain ne dépend d'aucun framework
- [ ] Les Use Cases sont testés **sans** DB
- [ ] La couche Data implémente des interfaces/contrats définis par le Domain
- [ ] La couche Presentation appelle **uniquement** les use cases
- [ ] On peut **échanger** la Data layer sans toucher au Domain
